# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import json
import requests
import csv
import numpy as np
import pandas as pd
import joblib


import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.automl.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.44.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
%%file training/train.py

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from azureml.core import Workspace, Dataset
import argparse
from azureml.core.run import Run


def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

    args = parser.parse_args()

    run = Run.get_context()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))

    # Load dataset
    subscription_id = '4999417e-f032-4d4c-982a-c229e26aa825'
    resource_group = 'udacity-learning'
    workspace_name = 'udacity'

    workspace = Workspace(subscription_id, resource_group, workspace_name)

    dataset = Dataset.get_by_name(workspace, name='heart_failure_dataset')
    df = dataset.to_pandas_dataframe()
    
    X, y = df.iloc[:,:-1], df['DEATH_EVENT']

    model = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(class_weight='balanced', C=args.C, max_iter=args.max_iter))
        ])

    auc_scorer = make_scorer(roc_auc_score, greater_is_better=True)
    score = cross_val_score(model, X, y, cv=4, n_jobs=-1, scoring=auc_scorer)
    run.log("AUC_weighted", np.float(np.mean(score)))


if __name__ == '__main__':
    main()

Overwriting training/train.py


## Create or Attach an AmlCompute cluster
You will need to create a compute target for your AutoML run. 

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity
udacity-learning
germanywestcentral
4999417e-f032-4d4c-982a-c229e26aa825


In [4]:
# Create the CPU computer cluster
amlcompute_cluster_name = "capstone"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=5,
                                                           min_nodes=0,
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [5]:
# Choose a name for the run history container in the workspace
experiment_name = 'heart-failure'
experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Hyperdrive Configuration

Here I am using a simple Logistic Regression model to classify heart failure cases.

- For early termination policy I have chosen BanditPolicy with a slack factor of 0.1, which means if the metric falls by 10% the termination policy activates, and it also avoids premature termination in the first 5 runs

- For parameter sampling, I choose random parameter sampling and sample two hyperparameters C (regularization strength) and max_iter (maximum number of iterations)

- primary metric name for our algorithm is AUC_weighted, but named as Accuracy here (in the training script I used the right scorer algorithm). The training script can be found under training/train.py and do a hyperparam search for 20 runs


In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5) 

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--C': uniform(-0.01, 1.0),
    '--max_iter': choice(50, 100, 150)
})

# Setup environment for your training run
ml_env = Environment.from_conda_specification(name = 'ml-env', file_path = './training/conda_env.yml')

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(
    source_directory='./training/',
    script='train.py',
    compute_target=amlcompute_cluster_name,
    environment=ml_env,
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='AUC_weighted',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)



In [7]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()#
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_821a448c-76db-4d86-be95-cda84d54e25d
Web View: https://ml.azure.com/runs/HD_821a448c-76db-4d86-be95-cda84d54e25d?wsid=/subscriptions/4999417e-f032-4d4c-982a-c229e26aa825/resourcegroups/udacity-learning/workspaces/udacity&tid=fd3b6086-53e5-43ac-9b6b-b17e80f6e2f7

Streaming azureml-logs/hyperdrive.txt

[2022-11-10T22:42:32.624384][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-10T22:42:33.4613664Z][SCHEDULER][INFO]Scheduling job, id='HD_821a448c-76db-4d86-be95-cda84d54e25d_0' 
[2022-11-10T22:42:33.5862096Z][SCHEDULER][INFO]Scheduling job, id='HD_821a448c-76db-4d86-be95-cda84d54e25d_1' 
[2022-11-10T22:42:33.7035844Z][SCHEDULER][INFO]Scheduling job, id='HD_821a448c-76db-4d86-be95-cda84d54e25d_2' 
[2022-11-10T22:42:33.8033450Z][SCHEDULER][INFO]Scheduling job, id='HD_821a448c-76db-4d86-be95-cda84d54e25d_3' 
[2022-11-10T22:42:33.767891][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-11-10T22

{'runId': 'HD_821a448c-76db-4d86-be95-cda84d54e25d',
 'target': 'capstone',
 'status': 'Completed',
 'startTimeUtc': '2022-11-10T22:42:31.906084Z',
 'endTimeUtc': '2022-11-10T22:57:21.344289Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC_weighted","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '83fbc9ef-1aa9-4781-8948-830a7d835d08',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.7248774509803921',
  'best_child_run_id': 'HD_821a448c-76db-4d86-be95-cda84d54e25d_8',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_821a448c-76db-4d86-be95-cda84d54e25d_8'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVal

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
best_run_metric = hyperdrive_run.get_best_run_by_primary_metric()
best_metrics = best_run_metric.get_metrics()
print('\ROC_AUC_Score:', best_metrics['AUC_weighted'])
print(best_run_metric.get_file_names())
print('Best Run: ', best_run_metric)

\ROC_AUC_Score: 0.7248774509803921
['logs/azureml/dataprep/0/rslex.log.2022-11-10-22', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
Best Run:  Run(Experiment: heart-failure,
Id: HD_821a448c-76db-4d86-be95-cda84d54e25d_8,
Type: azureml.scriptrun,
Status: Completed)


In [ ]:
#TODO: Save the best model
model = best_run_metric.register_model(model_name='best_run_metric', model_path='/outputs/best_hyperdrive_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

